<a href="https://colab.research.google.com/github/MockaWolke/Places/blob/master/Transfer_train_all_further.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
model_name = "ENB2-Aug-Transfer-All"

In [ ]:
!git clone https://github.com/MockaWolke/Places.git
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/UAL/View_Directions.zip View_Directions.zip 
!cp drive/MyDrive/UAL/data.csv Places/data.csv 
!unzip -q View_Directions.zip -d Places
!rm  View_Directions.zip

import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import os
os.chdir("Places")
import ds_generator
import models
import plotting

Cloning into 'Places'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 50 (delta 24), reused 36 (delta 11), pack-reused 0
Unpacking objects: 100% (50/50), done.
Mounted at /content/drive


In [ ]:
train_ds, val_ds = ds_generator.get_train_and_val()
model = tf.keras.models.load_model('../drive/MyDrive/UAL/models/ENB2-Aug-Transfer-All/last_model')
model = models.unfreeze_model(model)
model.load_weights("../drive/MyDrive/UAL/models/ENB2-Aug-Transfer-All/saves_after_e10")

In [ ]:
model_loc = "../drive/MyDrive/UAL/models/"+ model_name
log_dir = model_loc + "/logs"
checkpoint_path = model_loc + "/saves_after_e14/weights"
last_model = model_loc + "/last_model_after_e14"

if model_name not in os.listdir("../drive/MyDrive/UAL/models/"):

  os.makedirs(model_loc )
  os.makedirs(log_dir)
  os.makedirs(checkpoint_path)
  os.makedirs(last_model)
  


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.02,
    patience=4,
    verbose=1,
    mode='auto',
    restore_best_weights=True
) 

In [ ]:
hist = model.fit(train_ds, epochs=20, validation_data=val_ds,callbacks=[tensorboard_callback,cp_callback,early_stopping_callback],initial_epoch=14)

Epoch 15/20
   5/1616 [..............................] - ETA: 3:36:54 - loss: 0.1434 - accuracy: 0.9438

KeyboardInterrupt: ignored

In [ ]:
plotting.plot_hist(hist)

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
model.save(last_model)

KeyboardInterrupt: ignored

In [ ]:
test_ds = ds_generator.get_test_ds()